In [1]:
import os
import pandas as pd
import swisseph as swe
from swe_const import *
from vedicastro import VedicAstro as va

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
SWE_AYANAMSHAS = { "Krishnamurti" : swe.SIDM_KRISHNAMURTI, "Krishnamurti_Senthilathiban": swe.SIDM_KRISHNAMURTI_VP291,
                   "Lahiri": swe.SIDM_LAHIRI, "Lahiri_1940": swe.SIDM_LAHIRI_1940, "Lahiri_ICRC" : swe.SIDM_LAHIRI_ICRC,
                   "Lahiri_VP285" : swe.SIDM_LAHIRI_VP285, 
                }

In [4]:
pyswisseph_anayanamshas = {"SWE_CONST" : [], "AYANAMSHA" : []}
for nr in range(0, 48):
    pyswisseph_anayanamshas["SWE_CONST"].append(nr)
    pyswisseph_anayanamshas["AYANAMSHA"].append(swe.get_ayanamsa_name(sidmode = nr))
swe_ayans_df = pd.DataFrame(pyswisseph_anayanamshas)
swe_ayans_df.set_index('SWE_CONST', inplace=True)
swe_ayans_df

,AYANAMSHA
SWE_CONST,
0,Fagan/Bradley
1,Lahiri
2,De Luce
3,Raman
4,Usha/Shashi
5,Krishnamurti
6,Djwhal Khul
7,Yukteshwar
8,J.N. Bhasin


In [5]:
def utc_offset_str_to_float(utc_offset: str) -> float:
    hours, minutes = map(int, utc_offset.split(':'))
    return hours + minutes / 60.0 if utc_offset.startswith('+') else -1 * (abs(hours) + minutes / 60.0)

def decimal_to_dms(decimal_deg: float):
    deg, mins, sec, fractional_sec, sign_idx = swe.split_deg(decimal_deg, roundflag = swe.SPLIT_DEG_ZODIACAL )
    sec += fractional_sec  # Add fractional seconds to seconds
    return RASHIS[sign_idx], f"{deg:02d}:{mins:02d}:{int(sec):02d}"

def is_planet_retrograde(lonspeed : float):
    """ Returns if this object is direct, retrograde 
    or stationary. 
    
    """
    if abs(lonspeed) < 0.0003:
        return "Stationary"
    elif lonspeed > 0:
        return "Direct"
    else:
        return "Retrograde"
        
def get_rl_nl_sl_data(deg : float):
    """
    Returns the  Rashi (Sign) Lord, Nakshatra, Nakshatra Pada, Nakshatra Lord, Sub Lord and Sub Sub Lord 
    corresponding to the given degree.
    """
    duration = [7, 20, 6, 10, 7, 18, 16, 19, 17]

    lords = ["Ketu", "Venus", "Sun", "Moon", "Mars", "Rahu", "Jupiter", "Saturn", "Mercury"]

    star_lords = lords * 3 ## lords for the 27 Nakshatras

    ## Compute Sign lords
    sign_deg = deg % 360  # Normalize degree to [0, 360)
    sign_index = int(sign_deg // 30)  # Each zodiac sign is 30 degrees
    
    # Compute Nakshatra details
    nakshatra_deg = sign_deg % 13.332  # Each nakshatra is 13.332 degrees
    nakshatra_index = int(sign_deg // 13.332)  # Find the nakshatra index
    pada = int((nakshatra_deg % 13.332) // 3.325) + 1  # Each pada is 3.325 degrees

    # Ensure nakshatra_index is within bounds
    nakshatra_index = nakshatra_index % len(NAKSHATRAS)        

    # Compute SubLords
    deg = deg - 120 * int(deg / 120)
    degcum = 0
    i = 0

    while i < 9:
        deg_nl = 360 / 27
        j = i
        while True:
            deg_sl = deg_nl * duration[j] / 120
            k = j
            while True:
                deg_ss = deg_sl * duration[k] / 120
                degcum += deg_ss
                if degcum >= deg:
                    return {"Nakshatra": NAKSHATRAS[nakshatra_index] + "-" + str(pada), 
                            "RasiLord": SIGN_LORDS[sign_index], "NakshatraLord": star_lords[nakshatra_index],
                            "SubLord": lords[j], "SubSubLord": lords[k] }
                k = (k + 1) % 9
                if k == j:
                    break
            j = (j + 1) % 9
            if j == i:
                break
        i += 1  

# Display them side by side with titles using HTML display with formatted string
def display_side_by_side(df1_html: str, df2_html: str, localized_time: str, latitude: float, longitude: float,
                         heading1: str, heading2 : str):
    html = f"""
    <b>Planet Position Results for :</b> {localized_time} at <b>Lat:</b>{latitude} | <b>Long:</b> {longitude}
    <div style='display:flex; justify-content:space-between;'>
        <div style='width: 45%;'>
            <h2>{heading1}</h2>
            {df1_html}
        </div>
        <div style='width: 50%;'>
            <h2>{heading2}</h2>
            {df2_html}
        </div>
    </div>
    """
    display(HTML(html))        

In [6]:
## Case 2
year = 2000
month = 12
day = 30
hour = 22
minute = 31
secs = 59
latitude, longitude, utc_offset = 28.6334, 77.2834, "+5:30" ## Delhi

In [7]:
from pytz import timezone
from datetime import datetime
zone = "Asia/Calcutta"
### TIMEZONE INFORMATION
tz = timezone(zone)
localized_time = tz.localize(datetime(year, month, day, hour, minute, secs))

In [8]:
utc_float =  utc_offset_str_to_float(utc_offset)

utc = swe.utc_time_zone(year, month, day, hour = hour, minutes = minute, seconds = secs, offset = utc_float)
_ , jd_ut_start = swe.utc_to_jd(*utc) ## Unpacks utc tuple
current_time = jd_ut_start

### Assessing various values for `flags` arg

In [9]:
print(swe.FLG_SWIEPH)
print(swe.FLG_SIDEREAL)
print(swe.get_ayanamsa_ut(current_time))
print(swe.get_ayanamsa_ex_ut(current_time, flags = swe.FLG_SWIEPH))
print(swe.get_ayanamsa_ex_ut(current_time, flags = swe.FLG_SIDEREAL | swe.FLG_SPEED))
print(swe.get_ayanamsa_ex_ut(current_time, flags = swe.FLG_SIDEREAL + swe.FLG_SWIEPH ))
print(swe.get_ayanamsa_ex_ut(current_time, flags = swe.FLG_SWIEPH | swe.FLG_SIDEREAL)[1])

2
65536
24.754229119087938
(2, 24.749758266548277)
(2, 24.749758266548277)
(2, 24.749758266548277)
24.749758266548277


In [10]:
# Assuming SWE_AYANAMSHAS and other necessary imports and variables are already defined
multi_eph_data = {}
swe.set_ephe_path('/Users/dilip.rajkumar/Documents/jpl_eph_data')
for flg_type in ["swe_eph","jpl_eph_de406e","jpl_eph_441"]:
    # Flag Setting
    if flg_type == "swe_eph":
        flg = swe.FLG_SWIEPH | swe.FLG_SIDEREAL | swe.FLG_SPEED
    else:
        flg =  swe.FLG_JPLEPH | swe.FLG_SIDEREAL | swe.FLG_SPEED
        if flg_type == "jpl_eph_de406e":
            swe.set_jpl_file("de406e.eph")
        elif flg_type == "jpl_eph_441":
            swe.set_jpl_file("de441.eph")

    multi_eph_data[flg_type] = {}
    for ayan in list(SWE_AYANAMSHAS.keys()):
        # print(f"\nCalculating Planetary Position Data for {flg_type} and {ayan} Ayanamsha")
        swe.set_sid_mode(SWE_AYANAMSHAS.get(ayan))  # Set the ayanamsa based on the current key
        planetary_positions = []
        for name, body in PLANETS.items():
            result, retflag = swe.calc_ut(current_time, body, flags =  flg)
            # if name in ["Sun","Moon"]:print(f"Current EPH File Data in {ayan} ayanamsha for {name}:",swe.get_current_file_data(body))         
            long, lat, au_distance, speed = result[0], result[1], result[2], result[3]
            is_retrograde = is_planet_retrograde(speed)
            long = (long + 180) % 360 if name == "Ketu" else long
            sign, sign_dms = decimal_to_dms(long)
            planet_details = {"Object": name, "Rasi":sign, "isRetroGrade":is_retrograde, "LonDecDeg": round(long, 4), "SignLonDMS": sign_dms} # "Lat": lat, "AU_distance": au_distance,
            planet_details.update(get_rl_nl_sl_data(deg = long))
            planetary_positions.append(planet_details)

        multi_eph_data[flg_type][ayan] = planetary_positions

In [11]:
def generate_eph_data_comparisons(ayanamsha_selected: str):
    eph_data_comparisons = {
        "Planets": list(PLANETS.keys()),
        "Ayanamsha": [ayanamsha_selected] * len(list(PLANETS.keys())),
        "Swiss_sepl18.eph_semo18.eph": pd.DataFrame(multi_eph_data["swe_eph"][ayanamsha_selected]).LonDecDeg.tolist(),
        "JPL_de441.eph": pd.DataFrame(multi_eph_data["jpl_eph_441"][ayanamsha_selected]).LonDecDeg.tolist(),
        "JPL_de406e.eph": pd.DataFrame(multi_eph_data["jpl_eph_de406e"][ayanamsha_selected]).LonDecDeg.tolist()
    }
    return eph_data_comparisons

print(f"EPH-wise Planetary Positions ")
df_temp_ayan1 = pd.DataFrame(generate_eph_data_comparisons(ayanamsha_selected="Krishnamurti_Senthilathiban"))
df_temp_ayan2 = pd.DataFrame(generate_eph_data_comparisons(ayanamsha_selected="Krishnamurti"))
display_side_by_side(df_temp_ayan1.to_html(), df_temp_ayan2.to_html(), localized_time, latitude, longitude,
                      heading1 = "", heading2 = "")



EPH-wise Planetary Positions 


,Planets,Ayanamsha,Swiss_sepl18.eph_semo18.eph,JPL_de441.eph,JPL_de406e.eph
0,Sun,Krishnamurti_Senthilathiban,255.5283,255.5283,255.5283
1,Moon,Krishnamurti_Senthilathiban,309.4652,309.4652,309.4652
2,Mercury,Krishnamurti_Senthilathiban,258.3928,258.3928,258.3928
3,Venus,Krishnamurti_Senthilathiban,301.7480,301.7480,301.7480
4,Mars,Krishnamurti_Senthilathiban,190.3926,190.3926,190.3926
5,Jupiter,Krishnamurti_Senthilathiban,38.5065,38.5065,38.5065
6,Saturn,Krishnamurti_Senthilathiban,30.8583,30.8583,30.8583
7,Uranus,Krishnamurti_Senthilathiban,294.7980,294.7980,294.7980
8,Neptune,Krishnamurti_Senthilathiban,281.4922,281.4922,281.4922
9,Pluto,Krishnamurti_Senthilathiban,229.9324,229.9324,229.9324


## FlatLib Planetary Results

In [12]:
vhd = va.VedicHoroscopeData(year = year, month = month, day = day, hour = hour, minute = minute, second = secs, utc = utc_offset, 
                        latitude = latitude, longitude = longitude, ayanamsa = "Krishnamurti_Senthilathiban", house_system = "Placidus")

chart = vhd.generate_chart()
planets_data = vhd.get_planets_data_from_chart(chart)

## Filter Planets Data
planets_data_filtered = [planet for planet in planets_data if planet.Object not in ["Asc","Chiron", "Syzygy", "Fortuna"]]

df_flatlib_planets = pd.DataFrame(planets_data_filtered)
df_flatlib_planets

,Object,Rasi,isRetroGrade,LonDecDeg,SignLonDMS,SignLonDecDeg,LatDMS,Nakshatra,RasiLord,NakshatraLord,SubLord,SubSubLord,HouseNr
0,Sun,Sagittarius,False,255.528,+15:31:42,15.528,+01:01:10,PurvaAshadha,Jupiter,Venus,Venus,Ketu,4
1,Moon,Aquarius,False,309.465,+09:27:55,9.465,+11:52:57,Shatabhisha,Saturn,Rahu,Jupiter,Ketu,6
2,Mercury,Sagittarius,False,258.393,+18:23:34,18.393,+01:36:56,PurvaAshadha,Jupiter,Venus,Rahu,Rahu,4
3,Venus,Aquarius,False,301.748,+01:44:53,1.748,+01:06:36,Dhanishta,Saturn,Mars,Mercury,Saturn,6
4,Mars,Libra,False,190.393,+10:23:33,10.393,+00:35:10,Svati,Venus,Rahu,Jupiter,Rahu,2
5,Jupiter,Taurus,True,38.506,+08:30:23,8.506,-00:05:03,Krittika,Venus,Sun,Venus,Mars,9
6,Saturn,Taurus,True,30.858,+00:51:30,0.858,-00:02:43,Krittika,Venus,Sun,Rahu,Sun,9
7,Uranus,Capricorn,False,294.798,+24:47:53,24.798,+00:02:52,Dhanishta,Saturn,Mars,Rahu,Saturn,6
8,Neptune,Capricorn,False,281.492,+11:29:32,11.492,+00:02:05,Shravana,Saturn,Moon,Mars,Saturn,5
9,Pluto,Scorpio,False,229.932,+19:55:57,19.932,+00:02:09,Jyeshtha,Mars,Mercury,Venus,Moon,4


In [13]:
df_swe_planets = pd.DataFrame(multi_eph_data["swe_eph"]["Krishnamurti_Senthilathiban"])
df_swe_planets = df_swe_planets.round({'LonDecDeg': 3})

### Comparison of `flatlib` vs `pyswisseph` results for `Krishnamurti_Senthilathiban` Ayanamsha

In [14]:
final_cols = ["Object","Rasi","LonDecDeg","SignLonDMS"]
df1_html = df_swe_planets[final_cols].to_html()
df2_html = df_flatlib_planets[final_cols].to_html()

In [15]:
display_side_by_side(df1_html, df2_html, localized_time, latitude, longitude, 
                     heading1 = "Swisseph - Planetary Results", heading2 = "Flatlib - Planetary Results")

,Object,Rasi,LonDecDeg,SignLonDMS
0,Sun,Sagittarius,255.528,15:31:41
1,Moon,Aquarius,309.465,09:27:54
2,Mercury,Sagittarius,258.393,18:23:34
3,Venus,Aquarius,301.748,01:44:52
4,Mars,Libra,190.393,10:23:33
5,Jupiter,Taurus,38.506,08:30:23
6,Saturn,Taurus,30.858,00:51:29
7,Uranus,Capricorn,294.798,24:47:52
8,Neptune,Capricorn,281.492,11:29:32
9,Pluto,Scorpio,229.932,19:55:56
